# <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Welcome to Rideshare AI Lakehouse powered by BigQuery Studio

## **Step 01: Overview**

This portion of the demo will showcase how you can build an AI Lakehouse using Google Cloud's Data and Analytics stack.  In this notebook we will match prefered drivers to customers based on their preferences.

All code is located on GitHub: https://goo.gle/dagd

Please check out the other part of the demo which uses AI, image object analysis and data at scale.

This the steps we will perform:
- Generate a semantic embedding using the  `gecko` model for the column `llm_customer_quantitative_analysis`on the table `customer`, this column contains a LLM based summarization of the customer preferences. This is done using the BQML function `ML.GENERATE_TEXT_EMBEDDING`
- Generate a semantic embedding using the  `gecko` model for the column `llm_driver_quantitative_analysis`on the table `driver`, this column contains a LLM based summarization of the driver preferences. This is done using the BQML function `ML.GENERATE_TEXT_EMBEDDING`
- For each customer, find the most relevant drivers, three in thos case, this is done searching for the "closests" embeddings between the two tables. This is done using the BQML function `ML.DISTANCE`


## **Step 02: Generate semantic embeddings**

Next, we will use the  Vertex AI `textembedding-gecko` foundation model with the ML.GENERATE_TEXT_EMBEDDING function to embed the customer and driver preferences

Create embeddings for customer preferences

In [10]:
%%bigquery
UPDATE `${project_id}.${bigquery_rideshare_llm_curated_dataset}.customer` AS customer
  SET llm_customer_quantitative_analysis_embedding = child.text_embedding
  FROM (SELECT customer_id,text_embedding FROM ML.GENERATE_TEXT_EMBEDDING(
    MODEL rideshare_llm_curated.llm_embedding_model,
    (SELECT customer_id,customer_quantitative_analysis as content FROM `${project_id}.${bigquery_rideshare_llm_curated_dataset}.customer` WHERE include_in_llm_processing = TRUE),
    STRUCT(TRUE AS flatten_json_output)
  )) AS child
WHERE customer.customer_id = child.customer_id

Query is running:   0%|          |

""


In [12]:
%%bigquery
SELECT customer_id
FROM `${project_id}.${bigquery_rideshare_llm_curated_dataset}.customer`
WHERE include_in_llm_processing = TRUE
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,customer_id
0,3184
1,4027
2,1745
3,240
4,387


Lets inspect just one customer record

In [13]:
customer_id = 4027 #@param{type:"number"}
params = { "customer_id": customer_id}

In [14]:
%%bigquery --params $params
SELECT customer_id,customer_quantitative_analysis,llm_customer_quantitative_analysis_embedding
FROM `${project_id}.${bigquery_rideshare_llm_curated_dataset}.customer`
WHERE customer_id = @customer_id;

Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,customer_quantitative_analysis,llm_customer_quantitative_analysis_embedding
0,4027,Rodolfo Milanesi is a weekday rush-hour commut...,"[0.013938041403889656, -0.020008759573101997, ..."


Create embeddings for driver preferences

In [15]:
%%bigquery
UPDATE `${project_id}.${bigquery_rideshare_llm_curated_dataset}.driver` AS driver
  SET llm_driver_quantitative_analysis_embedding = child.text_embedding
  FROM (SELECT driver_id,text_embedding FROM ML.GENERATE_TEXT_EMBEDDING(
    MODEL rideshare_llm_curated.llm_embedding_model,
    (SELECT driver_id,driver_quantitative_analysis as content FROM `${project_id}.${bigquery_rideshare_llm_curated_dataset}.driver` WHERE include_in_llm_processing = TRUE),
    STRUCT(TRUE AS flatten_json_output)
  )) AS child
WHERE driver.driver_id = child.driver_id

Query is running:   0%|          |

""


In [17]:
%%bigquery
SELECT driver_id
FROM `${project_id}.${bigquery_rideshare_llm_curated_dataset}.driver`
WHERE include_in_llm_processing = TRUE
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,driver_id
0,17
1,55
2,29
3,79
4,80


Lets inspect just one driver record

In [18]:
driver_id = 29 #@param{type:"number"}
params = { "driver_id": driver_id}

In [19]:
%%bigquery --params $params
SELECT driver_id,driver_quantitative_analysis,llm_driver_quantitative_analysis_embedding
FROM `${project_id}.${bigquery_rideshare_llm_curated_dataset}.driver`
WHERE driver_id = @driver_id;

Query is running:   0%|          |

Downloading:   0%|          |

,driver_id,driver_quantitative_analysis,llm_driver_quantitative_analysis_embedding
0,29,The driver picks up customers from many pickup...,"[-0.01263249572366476, -0.03208749368786812, -..."


 ## Step 03: Perform the top-K embeddings similarity matching

In [20]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

In [21]:
batch_size = 10
update_sql="""
FOR customer_embedding_data IN (SELECT customer_id,llm_customer_quantitative_analysis_embedding FROM  `${project_id}.${bigquery_rideshare_llm_curated_dataset}.customer` WHERE ARRAY_LENGTH(llm_customer_quantitative_analysis_embedding) !=0 AND ARRAY_LENGTH(prefered_drivers) = 0 LIMIT {batch_size} )
DO
UPDATE `${project_id}.${bigquery_rideshare_llm_curated_dataset}.customer` AS customer
  SET prefered_drivers = child.matched_drivers
  FROM (
        WITH top_drivers AS
            (SELECT AS STRUCT
                customer_embedding_data.customer_id,
                customer_embedding_data.llm_customer_quantitative_analysis_embedding,
                driver.llm_driver_quantitative_analysis_embedding,
                driver.driver_id,
                ML.DISTANCE(customer_embedding_data.llm_customer_quantitative_analysis_embedding, driver.llm_driver_quantitative_analysis_embedding, 'COSINE') AS distance
            FROM
                `${project_id}.${bigquery_rideshare_llm_curated_dataset}.driver` AS driver
            WHERE
                ARRAY_LENGTH(driver.llm_driver_quantitative_analysis_embedding) !=0
            ORDER BY distance ASC
            --top 3 matches
          LIMIT 3)
      SELECT ARRAY_AGG(driver_id) matched_drivers FROM top_drivers) AS child
  WHERE customer.customer_id = customer_embedding_data.customer_id;
END FOR;
  """.format(batch_size=batch_size)

print("SQL: {update_sql}".format(update_sql=update_sql))

SQL: 
FOR customer_embedding_data IN (SELECT customer_id,llm_customer_quantitative_analysis_embedding FROM  `rideshare_llm_curated.customer` WHERE ARRAY_LENGTH(llm_customer_quantitative_analysis_embedding) !=0 AND ARRAY_LENGTH(prefered_drivers) = 0 LIMIT 10 )
DO
UPDATE `rideshare_llm_curated.customer` AS customer
  SET prefered_drivers = child.matched_drivers
  FROM (
        WITH top_drivers AS
            (SELECT AS STRUCT
                customer_embedding_data.customer_id,
                customer_embedding_data.llm_customer_quantitative_analysis_embedding,
                driver.llm_driver_quantitative_analysis_embedding,
                driver.driver_id,
                ML.DISTANCE(customer_embedding_data.llm_customer_quantitative_analysis_embedding, driver.llm_driver_quantitative_analysis_embedding, 'COSINE') AS distance
            FROM
                rideshare_llm_curated.driver AS driver
            WHERE
                ARRAY_LENGTH(driver.llm_driver_quantitative_analysis_e

In [22]:
import time

done = False
displayed_first_sql = False
original_record_count = 0

while done == False:
  # Get the count of records to match
  sql = """
        SELECT COUNT(*) AS cnt
          FROM `${project_id}.${bigquery_rideshare_llm_curated_dataset}.customer` customer
         WHERE
    ARRAY_LENGTH(customer.llm_customer_quantitative_analysis_embedding) !=0 AND ARRAY_LENGTH(customer.prefered_drivers) = 0  ;
        """

  df_record_count = client.query(sql).to_dataframe()
  cnt = df_record_count['cnt'].head(1).item()
  if displayed_first_sql == False:
    original_record_count = cnt
    displayed_first_sql = True

  print("Remaining records to process: ", cnt, " out of", original_record_count, " batch_size: ", batch_size)


  if cnt == 0:
    done = True
  else:
    # https://github.com/googleapis/python-bigquery/tree/master/samples
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(update_sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))

    while query_job.state != "DONE":
      time.sleep(5)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))

Remaining records to process:  100  out of 100  batch_size:  10
Job ea7a4f42-b3a4-4351-baa2-56999950245a is currently in state RUNNING
Job ea7a4f42-b3a4-4351-baa2-56999950245a is currently in state RUNNING
Job ea7a4f42-b3a4-4351-baa2-56999950245a is currently in state RUNNING
Job ea7a4f42-b3a4-4351-baa2-56999950245a is currently in state RUNNING
Job ea7a4f42-b3a4-4351-baa2-56999950245a is currently in state RUNNING
Job ea7a4f42-b3a4-4351-baa2-56999950245a is currently in state RUNNING
Job ea7a4f42-b3a4-4351-baa2-56999950245a is currently in state RUNNING
Job ea7a4f42-b3a4-4351-baa2-56999950245a is currently in state DONE
Remaining records to process:  90  out of 100  batch_size:  10
Job 3242bb10-9544-4046-80e0-6810a9a7a034 is currently in state RUNNING
Job 3242bb10-9544-4046-80e0-6810a9a7a034 is currently in state RUNNING
Job 3242bb10-9544-4046-80e0-6810a9a7a034 is currently in state RUNNING
Job 3242bb10-9544-4046-80e0-6810a9a7a034 is currently in state RUNNING
Job 3242bb10-9544-4046-8

Lets check one matched customer-driver pair

In [23]:
%%bigquery
SELECT customer_id FROM `${project_id}.${bigquery_rideshare_llm_curated_dataset}.customer` customer
WHERE ARRAY_LENGTH(customer.llm_customer_quantitative_analysis_embedding) !=0 AND ARRAY_LENGTH(customer.prefered_drivers) != 0
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,customer_id
0,1745
1,4027
2,3184
3,387
4,240


In [24]:
customer_id = 4027 #@param{type:"number"}
params = { "customer_id": customer_id}

In [26]:
%%bigquery --params $params
SELECT driver_quantitative_analysis,customer_quantitative_analysis
FROM
    `${project_id}.${bigquery_rideshare_llm_curated_dataset}.customer` AS customer,
    `${project_id}.${bigquery_rideshare_llm_curated_dataset}.driver` AS driver
WHERE driver.driver_id = customer.prefered_drivers[0]
AND customer.customer_id = @customer_id;

Query is running:   0%|          |

Downloading:   0%|          |

,driver_quantitative_analysis,customer_quantitative_analysis
0,The driver picks up customers from many locati...,Rodolfo Milanesi is a weekday rush-hour commut...
